# Fast Break
We want to engineer the feature that considers if a fast break is happening?
If a fast break is happening, we expect the XG will increase.
## What hueristic are we using to determine a fast break?  
A fast break is started when a ball is stolen then the ball ends up in the opponents final 3rd. We can look at speed of players and ball. For now, if the ball is stolen and put near the opponents goal that is a fast break.

In [12]:
# Add this so we can import our haxml code from outside the notebooks folder.
import sys
sys.path.append("../")

from haxml.prediction import (
    generate_rows_demo
)
from haxml.utils import (
    load_match,
    inflate_match,
    get_stadiums,
    get_matches_metadata,
    to_clock,
    train_test_split_matches_even_count
)
from haxml.viz import (
    plot_stadium,
    zoom_stadium,
    plot_positions
)
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import json
import pandas as pd
from tqdm import tqdm

# should move this to a haxml library i.e utils
def make_df(metadata, callback, progress=False):
    """
    Transforms match metadata into a DataFrame of records for
    each kick, including target label and features.
    Args:
        metadata: Match metadata (list of dicts).
        callback: Method to run on each match to extract kicks.
        progress: Whether or not to show progress bar (boolean).
    Returns:
        DataFrame where each row is a kick record.
    """
    rows = []
    bar = tqdm(metadata) if progress else metadata
    for meta in bar:
        key = meta["match_id"]
        infile = "../data/packed_matches/{}.json".format(key)
        try:
            s = stadiums[meta["stadium"]]
            row_gen = load_match(infile, lambda m: callback(m, s))
            for row in row_gen:
                row["match"] = key
                rows.append(row)
        except FileNotFoundError:
            pass
    return pd.DataFrame(rows)

In [13]:
stadiums = get_stadiums("../data/stadiums.json")
metadata = get_matches_metadata("../data/matches_metadata.csv")

In [14]:
meta = metadata[321]
key = meta["match_id"]
infile = "../data/packed_matches/{}.json".format(key)
stadium = stadiums[meta["stadium"]]
match = load_match(infile)
row_gen = generate_rows_demo(match, stadium)
df = pd.DataFrame(row_gen)
df["match"] = key

In [40]:
def isFastBreak(kick):
    '''
    Takes in a kick and where it ends up 
    '''
    travel_dist = ((kick['fromX'] - kick['toX'])**2 + (kick['fromY'] - kick['toY'])**2)**(1/2)
    if travel_dist > 200 and kick['type'] == "steal":
        return True
    return False

In [41]:
# can probably find a way to write less code
def get_defender_features(match):
    """
    Args:
        match: index for match within metadata
    Returns:
        Return a df containing new features for fast break
        fast break hueristic: steal and ball travels more than 300 dist
    """
    meta = get_matches_metadata("../data/matches_metadata.csv")[match]
    stadiums = get_stadiums("../data/stadiums.json")
    infile = "../data/packed_matches/{}.json".format(meta["match_id"])
    stadium = stadiums[meta["stadium"]]
    match = load_match(infile)
    row_gen = generate_rows_demo(match, stadium)
    df = pd.DataFrame(row_gen)
    kick_df = pd.DataFrame(match["kicks"]).to_dict(orient="records")
    is_fastBreak = []
    for kick in kick_df:
        is_fastBreak.append(isFastBreak(kick))
    df["FastBreak"] = is_fastBreak
    return df

In [42]:
new_df = get_defender_features(543)
new_df.head(20)

,ag,index,time,x,y,goal_x,goal_y,goal_distance,goal_angle,team,stadium,FastBreak
0,0,0,2.3,-12.0,15.0,700,0,712.157988,1.549732,red,NAFL Official Map v1,True
1,0,1,6.0,281.0,-108.0,-700,0,986.927049,1.461146,blue,NAFL Official Map v1,False
2,0,2,7.3,21.0,-276.0,-700,0,772.021373,1.205203,blue,NAFL Official Map v1,False
3,0,3,11.3,401.0,106.0,-700,0,1106.090864,1.474816,blue,NAFL Official Map v1,False
4,1,4,13.3,364.0,142.0,700,0,364.773903,1.170944,red,NAFL Official Map v1,False
5,0,5,17.8,20.0,-3.0,-700,0,720.006250,1.566630,blue,NAFL Official Map v1,False
6,0,6,17.8,-470.0,-17.0,700,0,1170.123498,1.556267,red,NAFL Official Map v1,False
7,0,7,23.7,-165.0,200.0,700,0,887.820365,1.343575,red,NAFL Official Map v1,True
8,0,8,26.1,343.0,-13.0,-700,0,1043.081013,1.558333,blue,NAFL Official Map v1,False
9,0,9,26.3,87.0,287.0,-700,0,837.698036,1.221107,blue,NAFL Official Map v1,False
